# Author : Nonbangkok
## Code for : https://www.kaggle.com/competitions/super-ai-ss-5-named-entity-recognition

### Preparation

In [ ]:
# !wget http://images.cocodataset.org/zips/train2017.zip
# !wget http://images.cocodataset.org/zips/val2017.zip


# !unzip train2017.zip
# !unzip val2017.zip

# !rm train2017.zip
# !rm val2017.zip

In [1]:
!pip install pillow
!pip install torch
!pip install transformers
!pip install googletrans==4.0.0-rc1
!pip install tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=02fe2bfed9ffea8823a8362d77eeb99e0e4ee50f3c8528423cb9b1e3d038ae4b
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
     

In [2]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00


### Captioning

In [3]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from deep_translator import GoogleTranslator
from PIL import Image
import pandas as pd
import numpy as np 
import torch
import json
import os

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
test_dir = "/kaggle/input/image-processing-thai-language-image-captioning/test/test"

In [10]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

result = []

for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames:
        image_path = os.path.join(dirname, filename)

        try:        
            with Image.open(image_path) as img:
                img = img.convert("RGB")
                img = img.resize((224, 224))
                
            inputs = processor(images=img, return_tensors="pt").to(device)
            
            with torch.no_grad():
                output = model.generate(**inputs)
            
            caption = processor.decode(output[0], skip_special_tokens=True)
            
            thai_caption = GoogleTranslator(source="en", target="th").translate(caption)
            
            result.append([filename, thai_caption])

            print(f"image : {filename}, caption : {thai_caption}")
        except:
            print("Error")

image : 00767.jpg, caption : มดดำ มด มด มด มด มด มด มด มด มด มด มด มด มด มด มด มด
image : 00266.jpg, caption : อาคารสีขาวมีป้ายเขียนว่า ' บ้านของพระเจ้า '
image : 01496.jpg, caption : ภาพระยะใกล้ของกองเศษไม้
image : 01600.jpg, caption : งูกำลังนอนอยู่บนพื้น
image : 00847.jpg, caption : ดอกไม้สีเหลืองในทุ่งหญ้า
image : 00822.jpg, caption : รูปปั้นเทพธิดาแห่งวัดฮินดู
image : 00614.jpg, caption : ทุ่งหญ้ากว้างใหญ่
image : 01850.jpg, caption : ต้นไม้ขนาดเล็กที่มีใบสีเขียวเติบโตบนพื้นดิน
image : 00878.jpg, caption : หนูตัวเล็ก ๆ นั่งอยู่บนโต๊ะ
image : 01473.jpg, caption : รูปปั้นขนาดใหญ่ในห้องที่มีคนจำนวนมาก
image : 00091.jpg, caption : เจ้าหน้าที่ตำรวจยืนอยู่หน้าอาคารที่พบศพชายคนหนึ่งในรถยนต์
image : 01713.jpg, caption : ชายสองคนกำลังทำงานบนโต๊ะไม้ในห้อง
image : 01306.jpg, caption : ชิ้นส่วนของหินบนโต๊ะ
image : 00838.jpg, caption : จานก๋วยเตี๋ยวและผักบนโต๊ะ
image : 01760.jpg, caption : ดอกไม้บนกิ่งไม้
image : 00296.jpg, caption : นกบินอยู่บนท้องฟ้าพร้อมกับว่าว
image : 01000.jpg, caption : 

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip2-opt-6.7b")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b").to(device)

test_dir = "/kaggle/input/image-processing-thai-language-image-captioning/test/test"

for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames:
        image_path = os.path.join(dirname, filename)
        
        try:        
            with Image.open(image_path) as img:
                img = img.convert("RGB")
                img = img.resize((224, 224))
                
            inputs = processor(images=img, return_tensors="pt").to(device)
            
            with torch.no_grad():
                output = model.generate(**inputs)
            
            caption = processor.decode(output[0], skip_special_tokens=True)
            
            thai_caption = GoogleTranslator(source="en", target="th").translate(caption)
            
            result.append([filename, thai_caption])

            print(f"image : {filename}, caption : {thai_caption}")
        except:
            print("Error")

In [11]:
len(result)

2000

In [12]:
result

[['00767.jpg', 'มดดำ มด มด มด มด มด มด มด มด มด มด มด มด มด มด มด มด'],
 ['00266.jpg', "อาคารสีขาวมีป้ายเขียนว่า ' บ้านของพระเจ้า '"],
 ['01496.jpg', 'ภาพระยะใกล้ของกองเศษไม้'],
 ['01600.jpg', 'งูกำลังนอนอยู่บนพื้น'],
 ['00847.jpg', 'ดอกไม้สีเหลืองในทุ่งหญ้า'],
 ['00822.jpg', 'รูปปั้นเทพธิดาแห่งวัดฮินดู'],
 ['00614.jpg', 'ทุ่งหญ้ากว้างใหญ่'],
 ['01850.jpg', 'ต้นไม้ขนาดเล็กที่มีใบสีเขียวเติบโตบนพื้นดิน'],
 ['00878.jpg', 'หนูตัวเล็ก ๆ นั่งอยู่บนโต๊ะ'],
 ['01473.jpg', 'รูปปั้นขนาดใหญ่ในห้องที่มีคนจำนวนมาก'],
 ['00091.jpg', 'เจ้าหน้าที่ตำรวจยืนอยู่หน้าอาคารที่พบศพชายคนหนึ่งในรถยนต์'],
 ['01713.jpg', 'ชายสองคนกำลังทำงานบนโต๊ะไม้ในห้อง'],
 ['01306.jpg', 'ชิ้นส่วนของหินบนโต๊ะ'],
 ['00838.jpg', 'จานก๋วยเตี๋ยวและผักบนโต๊ะ'],
 ['01760.jpg', 'ดอกไม้บนกิ่งไม้'],
 ['00296.jpg', 'นกบินอยู่บนท้องฟ้าพร้อมกับว่าว'],
 ['01000.jpg', 'ข้าวไก่กับผักหนึ่งจาน'],
 ['01445.jpg', 'ปลาที่มีสีแดงและสีขาวบนลำตัว'],
 ['01179.jpg', 'กระเบื้องสีฟ้าขาวลายมังกร'],
 ['00219.jpg', 'ภาพระยะใกล้ของพืชที่มีดิน'],
 ['01029.j

### Submission

In [13]:
submission = pd.read_csv("/kaggle/input/image-processing-thai-language-image-captioning/sample_submission.csv",dtype=str)

image_dict = dict(result)

for index, row in submission.iterrows():
    image_id = row['image_id']
    image_filename = str(image_id) + '.jpg'

    if image_id in ['01354.jpg', '01413.jpg', '01802.jpg']:
        continue
    
    if pd.isna(row['caption']) and image_filename in image_dict:
        submission.at[index, 'caption'] = image_dict[image_filename]

submission.to_csv('submission.csv', index=False, encoding='utf-8')
submission

,image_id,caption
0,01354,ภาพถ่ายระยะใกล้ของวัตถุทรงกลมสีขาวที่มีคราบสีด...
1,01413,นกตัวหนึ่งที่กําลังเกาะอยู่บนกิ่งไม้อันหนึ่งที...
2,01802,บ้านที่อยู่ติดกับบริเวณริมชายหาดทะเลและมีต้นไม...
3,01243,ลิงกำลังนั่งอยู่บนพื้นข้างกำแพง
4,00693,แมลงตัวเล็ก ๆ บนพื้นดินที่ปกคลุมด้วยมอส
...,...,...
1995,00029,บ่อน้ำขนาดใหญ่ในเรือนกระจกที่มีต้นไม้
1996,01065,ภาพระยะใกล้ของดอกไม้
1997,00195,วิวภูเขาจากเนินเขา
1998,01026,ดอกไม้ที่มีดอกสีเหลืองอยู่ด้านบน
